# California Housing Data

The task is to aproximate the median house value of each block from the rest of the variable

#### Import the housing.csv file with pandas .Seperate it into a traning (70%) and testing set(30%)

In [1]:
import pandas as pd

In [2]:
housing=pd.read_csv('housing.csv').drop(['longitude','latitude','ocean_proximity'],axis=1)

In [3]:
housing.head()

,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0


In [20]:
housing.dtypes

housing_median_age    float64
total_rooms           float64
total_bedrooms        float64
population            float64
households            float64
median_income         float64
median_house_value    float64
dtype: object

In [4]:
y_val=housing['median_house_value']

In [5]:
x_data=housing.drop(['median_house_value'],axis=1)

In [6]:
from sklearn.model_selection  import train_test_split

In [7]:
x_train,x_test,y_train,y_test=train_test_split(x_data,
                                             y_val,test_size=0.3,
                                              random_state=42)

# Scale the features

Use sklearn preprocessing to create a MinMaxScaler for the feature data.Fit this scaler 
only to  the training data.Then use it to transform x_test,and y_train.Then use the scaled x_test and x_train along with pd.Dataframe to re-create two dataframes of scaled data

In [8]:
from sklearn.preprocessing import MinMaxScaler

In [9]:
scaler=MinMaxScaler()

In [10]:
scaler.fit(x_train)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [11]:
x_train=pd.DataFrame(data=scaler.transform(x_train),
                     columns=x_train.columns,
                    index=x_train.index)

In [12]:
x_test=pd.DataFrame(data=scaler.transform(x_test),
                     columns=x_test.columns,
                    index=x_test.index)

# Create Feature Columns

#### Create the necessary tf.feature_column object for estimators They should all be trated as continious numeric_columns

In [13]:
housing.columns

Index(['housing_median_age', 'total_rooms', 'total_bedrooms', 'population',
       'households', 'median_income', 'median_house_value'],
      dtype='object')

In [14]:
import tensorflow as tf

In [15]:

age=tf.feature_column.numeric_column('housing_median_age')
rooms=tf.feature_column.numeric_column('total_rooms')
bedrooms=tf.feature_column.numeric_column('total_bedrooms')
pop=tf.feature_column.numeric_column('population')
household=tf.feature_column.numeric_column('households')
income=tf.feature_column.numeric_column('median_income')

In [16]:
feat_cols=[age,rooms,bedrooms,pop,household,income]

### Create the input function for estimator object(Play around with batch_size and num_epochs)

In [17]:
input_func=tf.estimator.inputs.pandas_input_fn(x=x_train,y=y_train,
                                              batch_size=10,
                                              num_epochs=1000,
                                              shuffle=True)

### Create a estimator model Use a DNNRegressior Play around with the hidden units!

In [18]:
dnn_model=tf.estimator.DNNClassifier(hidden_units=[10,10,10],
                                      feature_columns=feat_cols,
                                      n_classes=2)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\satya\\AppData\\Local\\Temp\\tmpow1hd57_', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001FC6DD0EFC8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


### Train the model for ~1000 steps(Later come back to this and train it for more and check for improvement)

In [19]:
dnn_model.train(input_fn=input_func,steps=1000)

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Savi

InvalidArgumentError: assertion failed: [Labels must <= n_classes - 1] [Condition x <= y did not hold element-wise:] [x (dnn/head/ToFloat:0) = ] [[70800][229800][500001]...] [y (dnn/head/assert_range/Const:0) = ] [1]
	 [[node dnn/head/assert_range/assert_less_equal/Assert/AssertGuard/Assert (defined at C:\Users\satya\AppData\Roaming\Python\Python37\site-packages\tensorflow_core\python\framework\ops.py:1748) ]]

### Create aprediction input function and then use the predict method off your estimator model to create a list or predictions on your test data

In [21]:
pred_fn=tf.estimator.inputs.pandas_input_fn(x=x_test,batch_size=len(x_test),shuffle=False)

In [23]:
pred_gn=dnn_model.predict(input_fn=pred_fn)

In [24]:
predictions=list(pred_gn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\satya\AppData\Local\Temp\tmpow1hd57_\model.ckpt-0
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


### Calculate the RMSE you should be able to get around 100000 RMSE(remember that this is in the units as the labels) do this manually or use sklearn.metrics

In [25]:
from sklearn.metrics import mean_squared_error

### in which value is not fix we fix accordin to our requirement

In [28]:
y_true = [3, -0.5, 2, 7]
y_pred = [2.5, 0.0, 2,8 ] 

In [29]:
mean_squared_error(y_true, y_pred)

0.375